In [1]:
# Libraries
import cobra
import pandas as pd
import numpy as np
import pyEQL.chemical_formula
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import sample
from  math import sqrt

#import csv
# function definitions
getSign=dict()

def getFBAvalues(model):
    ValFluxes=dict()
    fba = model.optimize()
    V=fba.fluxes
    for r in model.reactions:
        ValFluxes[r]= V.get(r.id)
    for r in model.reactions:
        if ValFluxes[r] > 0:
            getSign[r] = 1
        elif ValFluxes[r] < 0:
            getSign[r] = -1
        else:
            getSign[r] = 0
    return model.optimize()

def GetComposition_old(formula,atom="C"):
    ch=""
    flag=False
    for c in formula:
        if c == atom:
            flag=True
        else:
            if c.isnumeric() and flag:
                ch=ch+c
            else:
                flag=False
    if ch == "":
        return 0
    else:
        return int(ch)


def GetComposition(formula,atom="C"):
#    print("formula : ",formula)
    if (formula == None or formula == ""):
        formula = "S"
    try:
        return pyEQL.chemical_formula.get_element_mole_ratio(formula,atom)
    except ValueError:
        return 0
    
def getSign_j(r,fva=0):
    if fva == 0:
        minim=r.lower_bound
        maxim=r.upper_bound
    else:
        minim=fva["minimum"].get(r.id)
        maxim=fva["maximum"].get(r.id)
    if (abs(minim)+abs(maxim)>0.01):
        if (minim*maxim<0):
            return 0
        else:
            if (maxim<=0):
                return -1
            else:
                return 1
    else:
        return 0
            

ReactInputTable=dict()
ReactProductTable=dict()

def ReactionInputs(r,sign=1):
    if not ReactInputTable.get(r.id):
        ReactInputTable[r.id] = ReactionInputsLong(r,sign)
    return ReactInputTable.get(r.id)
 
        
def ReactionProducts(r,sign=1):
    if not ReactProductTable.get(r.id):
        ReactProductTable[r.id] = ReactionProductsLong(r,sign)
    return ReactProductTable.get(r.id)
    
def ReactionInputsLong(r,sign=1):
    res=list()
    for m in r.metabolites:
        if (sign*r.get_coefficient(m)<-0.01):
            res.append(m)
    return res
        
def ReactionProductsLong(r,sign=1):
    res=list()
    for m in r.metabolites:
        if (sign*r.get_coefficient(m)>0.01):
            res.append(m)
    return res
    

def In(r,m,atom="C"):
    res=0
    if m in ReactionProducts(r,getSign[r]):
        res=abs(r.get_coefficient(m))*GetComposition(m.formula,atom)
        somme=0
        for met in ReactionInputs(r,getSign[r]):
            somme+=abs(r.get_coefficient(met))*GetComposition(met.formula,atom)
        if not somme == 0:
            res=res/somme
        else:
            res=1
    return res


def RescaleModel(model,frac=0.5):
    fva=cobra.flux_analysis.flux_variability_analysis(model,fraction_of_optimum=frac).to_dict()
    for r in model.reactions:
        r.lower_bound = fva["minimum"].get(r.id)
        r.upper_bound = fva["maximum"].get(r.id)
        print("{} {:.3f} {:.3f}" .format(r.id, r.lower_bound, r.upper_bound))
        
        
    
def value_of_D(mk,mi,V,atom="C"):
    # case 1 : r is the exchange reactions for m input
    if mk == mi:
        return 1
    # General case
    res=0
    for r in isConsumedBy[mi.id]:
            mr=metRate(mi,V)
            if not mr==0: 
                res -= abs(r.get_coefficient(mi)) * V.get(r.id) *In(r,mk,atom) / mr
    return res 



def getMetabolitLists(model):
    LMet=list()
    for m in model.metabolites:
        LMet.append(m)
    LMetInputs=list()
    for m in LMet:
        flag=False
        for r in m.reactions:
            if len(ReactionInputs(r,getSign[r]))==0 and m in ReactionProducts(r,getSign[r]):
                flag=True
        if flag:
#            print(m.id,"is an input")
            LMetInputs.append(m)       
    LMetOutputs=list()
    for m in LMet:
        flag=False
        for r in m.reactions:
            if len(ReactionProducts(r,getSign[r]))==0 and m in ReactionInputs(r,getSign[r]):
                flag=True
        if flag:
#            print(m.id,"is an output")
            LMetOutputs.append(m)

    LMetOthers=list()
    for m in LMet:
        # Beurkkkk !!!!
        flag=True
        for r in m.reactions:
            if len(ReactionProducts(r,getSign[r]))==0 and m in ReactionInputs(r,getSign[r]):
                flag=False
            if len(ReactionInputs(r,getSign[r]))==0 and m in ReactionProducts(r,getSign[r]):
                flag=False
        if flag:
#            print(m.id,"is an neither an input, nor an output")
            LMetOthers.append(m)
    LMet = LMetInputs+LMetOthers+LMetOutputs
    return LMet,LMetInputs,LMetOthers,LMetOutputs

def getMatD(model,V,atom):
    # get the list of metabolites
    LMet, LIn, LOthers, LOut = getMetabolitLists(model)
    MatD=np.zeros((len(LMet),len(LMet)))
    i=0
    for mi in LMet:
        k=0
        for mk in LMet:
            MatD[k,i]=value_of_D(mk,mi,V,"C")
            k+=1
        i+=1
    return matD

def PrintRepartition(model,V,atom="C",aff=False,modelMamary=True, precision=1e-10):
    ReactInputTable.clear()
    ReactProductTable.clear()
    EnrichModel(model)
    # get the list of metabolites
    #res=dict()
    print("... Creating the metabolite lists")
    LMet, LIn, LOthers, LOut = getMetabolitLists(model)
    print("... Creating matrix D")
    MatD=np.zeros((len(LMet),len(LMet)))
    i=0
    for mi in LMet:
        k=0
        for mk in LMet:
            MatD[k,i]=value_of_D(mk,mi,V,atom)
            k+=1
        i+=1
    print("... Inverting matrix D")
    invmatD=np.linalg.inv(MatD)
    
    vect=list()
    D_LOut=len(LIn)+len(LOthers)
    for i in range(D_LOut, len(LMet)):
        ch = "X_"+LMet[i].id+"*("+str(GetComposition(LMet[i].formula,atom))+")"+" = "
        v=list()
        for k in range(len(LIn)):
            if (invmatD[i,k] > precision and (GetComposition(LMet[k].formula,atom))>0.01):
                ch += " + "+str((round(invmatD[i,k],3)) )+" * X_"+LMet[k].id+"("+str(GetComposition(LMet[k].formula,atom))+")"
                v.append((round( invmatD[i,k]*metRate(LIn[k],V)*GetComposition(LIn[k].formula,atom), 3) ))
            else:
                v.append(0)
        vect.append(v)
        if aff:
            print(ch)
    ar = np.array(vect)
    #print(vect)
    AIO_matrice = pd.DataFrame(ar)
    return  AIO_matrice


def Estimation_MinMax_AIO(model, sampling_V, readLastMinMAx=False):
    if readLastMinMAx:
        Min_AIO_matrice = pd.read_csv('MinAIO_FirstSample.csv', sep =';', header = None)
        Max_AIO_matrice = pd.read_csv('MaxAIO_FirstSample.csv', sep =';', header = None)
        Min_AIO_matrice_Pred = Min_AIO_matrice.copy() 
        Max_AIO_matrice_Pred = Max_AIO_matrice.copy() 
    else:   
        V=sampling_V.loc[0]
        Min_AIO_matrice=PrintRepartition(model,V)
        Max_AIO_matrice=Min_AIO_matrice.copy()
    Nblig_AIO_matrice=len(Min_AIO_matrice[0])
    Nbcol_AIO_matrice=len(Min_AIO_matrice.loc[0])
    for id_V in range (1,len(sampling_V.index)):
        V=sampling_V.loc[id_V]
        AIO_matrice=PrintRepartition(model,V) 
        for l in range(Nblig_AIO_matrice):   #Pour la colonne fixée je parcours les elts
            for c in range(Nbcol_AIO_matrice):
                if (AIO_matrice[c][l] < Min_AIO_matrice[c][l]):
                    Min_AIO_matrice[c][l] = AIO_matrice[c][l]
                if (AIO_matrice[c][l] > Max_AIO_matrice[c][l]):
                    Max_AIO_matrice[c][l] = AIO_matrice[c][l]
    if readLastMinMAx:
        return Min_AIO_matrice_Pred, Max_AIO_matrice_Pred, Min_AIO_matrice, Max_AIO_matrice
    else:
        return Min_AIO_matrice, Max_AIO_matrice

def Sum_EuclidianDist_ColonneMatriceAIO(AIO_matrice):
    Nblig_AIO_matrice=len(AIO_matrice[0])
    Nbcol_AIO_matrice=len(AIO_matrice.loc[0])
    Sum_EuclidianDist_Colonne=0
    for c in range(Nbcol_AIO_matrice):
        EuclidianDist_Colonne=0
        for l in range(Nblig_AIO_matrice):   #Pour la colonne fixée je parcours les elts
            EuclidianDist_Colonne+= AIO_matrice[c][l]*AIO_matrice[c][l]
        EuclidianDist_Colonne= sqrt(EuclidianDist_Colonne)
        Sum_EuclidianDist_Colonne+=EuclidianDist_Colonne
    return  Sum_EuclidianDist_Colonne


isProducedBy=dict()
isConsumedBy=dict()

def EnrichModel(model):
    isProducedBy.clear()
    isConsumedBy.clear()
    for m in model.metabolites:
        isProducedBy[m.id]=list()
        isConsumedBy[m.id]=list()
        for r in m.reactions:
            if m in ReactionProducts(r,getSign[r]):
                isProducedBy[m.id].append(r)
            if m in ReactionInputs(r,getSign[r]):
                isConsumedBy[m.id].append(r)

def metRate(m,V):
    res=0
    for r in isProducedBy[m.id]:
        res+=r.get_coefficient(m)*V.get(r.id)
    return res

def metRate_inv(m,V):
    res=0
    for r in m.reactions:
        if m not in ReactionProducts(r,getSign[r]):
            res+=abs(r.get_coefficient(m))*V.get(r.id)
    return res

def FixModelFluxes(model,V):
    for r in model.reactions:
        value = V.get(r.id)
        r.lower_bound = value
        r.upper_bound = value


In [2]:
#import cobra

from cobra import Model, Reaction, Metabolite
model = Model("Mammary Gland")
#AddInfoToMamarySbmlModel ajoute à chaque réaction du modèle un lower_bound et un upper_bound
#ajoute au modèle les formules biochimiques des métabolites Ici on considéré
#pour chaque métabolite une formule  sous forme Cn (on a pris uniquement en compte les carbones)
#AddInfoToMamarySbmlModel prend le fichier .csv des biologistes et retourne 
#des réactions de la aA+bB => cC+dD,  la liste des métabolites dans l'ordre
#du fichier et la liste correspondante au nombre de carbones de chaque métabolite


def AddInfoToMamarySbmlModel(InputFile_CSVNutrition, model, NumberLineRepartition):
    # Extraction de le 1ere et 5eme colonne et creation d'une matrice m lignes et 2 colonnes
    i=0
    Line=list()
    EquationsReactions=list() #sous la forme: aA+bB => cC+dD
    MesCol=list()
    MesColonnes=list()
    #cr = open("MamelleBis.csv","r")
    cr = open(InputFile_CSVNutrition,"r")
    for row in cr:
        t=row.replace("\n","").split(";")
        #print(t)
        Line.append(t)
        """
        if i > 0:
            MesCol=list()
            MesCol.append(t[0])
            MesCol.append(t[5])
            MesColonnes.append(MesCol)
        i=1
        """
    cr.close()
    #print(MesColonnes[0])
    #print(len(Line[0]))

    #On ajoute une 3eme colonne (reaction: A+B => C+D) à la matrice
    i=1
    while i < len(Line):
        cptRep=0
        gauche=""
        droite=""
        gaucheDroite=""
        j=6 #debut des solonnes métabolites
        while j < len(Line[i]):
            if float(Line[i][j]) == -1:
                gauche=gauche+ " "+Line[0][j]+" +"
            elif float(Line[i][j]) < 0:
                gauche=gauche+ " "+ str(abs(float(Line[i][j])))+ " "+Line[0][j]+" +"
            else:
                if float(Line[i][j]) == 1:
                    droite=droite+ " "+Line[0][j]+" +"
                elif float(Line[i][j]) > 0:
                    droite=droite+ " "+ str(abs(float(Line[i][j])))+ " "+Line[0][j]+" +"
            j+=1    
        gauche=gauche[0:-1]
        droite=droite[0:-1]
        gaucheDroite=gauche+" => "+ droite
        #MesColonnes[i-1].append(gaucheDroite)
        EquationsReactions.append(gaucheDroite)
        i+=1
        
    ListCarbonesMetabOrdreCSV=list()
    ListMetabOrdreCSV=list()
    j=6 #debut des solonnes métabolites
    while j < len(Line[0]):
        NbCarb="C"+str(int(float(Line[NumberLineRepartition][j]) ))
        ListCarbonesMetabOrdreCSV.append(NbCarb)
        ListMetabOrdreCSV.append(Line[0][j])
        j+=1
    """    
    f = open(OutputFile_CSVPython, 'w')
    for col in MesColonnes:
         ligne = ";".join(col) + "\n"
         f.write(ligne)
    f.close()
    """
    # ajoute à chaque réaction du modèle un lower_bound et un upper_bound
    i=1
    for r in  EquationsReactions:
        #print(r)
        React=Reaction("R"+str(i))
        React.name="R"+str(i)+": "+r
        #R=Reaction("R"+str(i))
        #R.name="R"+str(i)+": "+r
        #React.name="R"+str(i)
        model.add_reaction(React)
        model.reactions.get_by_id("R"+str(i)).build_reaction_from_string(r)
        model.reactions.get_by_id("R"+str(i)).lower_bound=0
        model.reactions.get_by_id("R"+str(i)).upper_bound=10000
        i+=1
    #ajoute au model sbml les formules biochimiques des métabolites
    #Ici on considéré uniquement les carbones (formule de chaque métabolite sous forme Cn)
    for m in model.metabolites:
        s=str(m)
        for n in ListMetabOrdreCSV:
            if(s==n):
                m.formula=ListCarbonesMetabOrdreCSV[ListMetabOrdreCSV.index(s)]
                break
                
    return    EquationsReactions, ListCarbonesMetabOrdreCSV, ListMetabOrdreCSV

#Cette fonction permet juste de réordonner les colonnes et les lignes d'une matrice AIO et avoir la matrice comme dans
#nutrition analyzer
def MamaryRepartitionOrder(model, AIO_matrice):
    LMet, LIn, LOthers, LOut = getMetabolitLists(model)
    cols = AIO_matrice.columns.tolist()
    new_col=[0, 2, 1, 12, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    new_index=[0, 11, 12, 1, 7, 8, 9, 6, 13, 14, 15, 2, 3, 4, 5, 16, 17, 10]
    #reordonner noms colonnes et lignes comme dans nutrition
    new_LIn=list()
    for val in new_col:
        new_LIn.append(LIn[val])   
    new_LOut=list()
    for val in new_index:
        new_LOut.append(LOut[val])


    AIO_matrice = AIO_matrice[new_col]
    AIO_matrice=AIO_matrice.reindex(new_index)

    AIO_matrice.index=new_LOut
    AIO_matrice.columns=new_LIn
    return  new_LOut, new_LIn, AIO_matrice


#appel de la fonction AddInfoToMamarySbmlModel
EquationsReactions, ListCarbMetOrdreCSV, ListMetOrdreCSV = AddInfoToMamarySbmlModel('MamelleBis.csv', model, 137)


unknown metabolite 'ATP' created
unknown metabolite 'ATP2' created
unknown metabolite 'GLC' created
unknown metabolite 'G6P' created
unknown metabolite 'lactose' created
unknown metabolite 'G3P' created
unknown metabolite 'NADHc' created
unknown metabolite 'PYR' created
unknown metabolite 'NADHm' created
unknown metabolite 'CO2' created
unknown metabolite 'ACoAm' created
unknown metabolite 'OAA' created
unknown metabolite 'aKG' created
unknown metabolite 'FADH2' created
unknown metabolite 'SERPivot' created
unknown metabolite 'NH3' created
unknown metabolite 'O2' created
unknown metabolite 'NADPH' created
unknown metabolite 'Lactate' created
unknown metabolite 'Acetate' created
unknown metabolite 'ACoAc' created
unknown metabolite 'Glycerol' created
unknown metabolite 'Lysine' created
unknown metabolite 'Methionine' created
unknown metabolite 'asparagine' created
unknown metabolite 'Cysteine' created
unknown metabolite 'Threonine' created
unknown metabolite 'Tryptophane' created
unknow

In [3]:

#print(EquationsReactions)




"""  
for m in model.metabolites:
    print(m)
    print(m.formula)
"""

'  \nfor m in model.metabolites:\n    print(m)\n    print(m.formula)\n'

In [4]:

# Remove unused reactions
ToBeRemoved=["R3","R12","R28","R31","R43","R45","R47","R48","R49","R53","R59","R60","R66","R69","R70","R73","R78"]
ToBeRemoved=ToBeRemoved+["R81","R84","R85","R57","R65"]
ToBeRemoved=ToBeRemoved+["R20","R21","R22","R23","R25","R32","R34","R38","R39","R42","R126","R129","R130","R132","R133","R137","R138"]
ToBeRemoved=ToBeRemoved+["R7","R50","R71","R123","R127"]
ToBeRemoved=ToBeRemoved+["R109","R110","R112","R116","R117"]


for r in ToBeRemoved:
    React=model.reactions.get_by_id(r)
    #print("Remove",r,":",React.build_reaction_string())
    React.remove_from_model()


# Remove unused metabolites
model.metabolites.ATP2.remove_from_model()
#model.metabolites.primer.remove_from_model()
#model.metabolites.Cn.remove_from_model()
model.metabolites.NH3.remove_from_model()
model.metabolites.O2.remove_from_model()





#model.reactions.Biomass.objective_coefficient=-1

biomass=Reaction("Biomass")
biomass.name="Biomass"
model.add_reaction(biomass)
model.reactions.Biomass.build_reaction_from_string("ATP =>")
biomass.lower_bound=0
biomass.upper_bound=10000




# Fix the input/output values

epsilon=0

Lvalues={
    "R2":237,
    "R95":5.84,
    "R96":510,
    "R97":84,
    "R98":0,
    "R108":2.68,
    "R111":0.35,
    "R113":2.19,
    "R114":2.02,
    "R115":2.54,
    "R118":0.23,
    "R119":4.4,
    "R128":3.11,
    "R131":1.22,
    "R24":0,
    "R76":0,
    "R83":0,
    "R62":32.96,
    "R99":73.8,
    "R100":10.08,
    "R101":4.51,
    "R102":2.23,
    "R103":4.66,
    "R104":4.23,
    "R105":13.9,
    "R106":18.82,
    "R107":124.5,
    "R120":4.98,
    "R121":0,
    "R122":0.54,
    "R124":10.65,
    "R125":3.43,
    "R134":0,
    "R136":7.21,
    "Biomass":1250
}


for r in Lvalues:
    React=model.reactions.get_by_id(r)
    React.lower_bound=Lvalues.get(r)-epsilon
    React.upper_bound=Lvalues.get(r)+epsilon
    


In [5]:
# test de calcul d'AIO d'un flux extémal
import time

globalstart = time.time()

start = time.time()

fba=getFBAvalues(model)
V=fba.fluxes
LMet,LMetInputs,LMetOthers,LMetOutputs=getMetabolitLists(model)


AIO_matrice=PrintRepartition(model,V)

new_LOut, new_LIn, AIO_matrice=MamaryRepartitionOrder(model, AIO_matrice)
print(AIO_matrice)

end = time.time()
print("       finishing in",end - start," s")



... Creating the metabolite lists
... Creating matrix D
... Inverting matrix D
                GLC  Glycerol  Acetate     BHBA  Lysine  Threonine  \
ATP           0.000     0.000    0.000    0.000   0.000      0.000   
Glycerol3P   94.235     4.645    0.000    0.000   0.000      0.000   
peptide       0.000     0.000    0.000    0.000   0.000      0.000   
lactose     885.600     0.000    0.000    0.000   0.000      0.000   
C4            0.000     0.000    0.000   40.320   0.000      0.000   
c6            0.000     0.000   18.040    9.020   0.000      0.000   
c8            0.000     0.000   13.380    4.460   0.000      0.000   
c10           0.000     0.000   37.280    9.320   0.000      0.000   
c12           0.000     0.000   42.300    8.460   0.000      0.000   
c14           0.000     0.000  166.800   27.800   0.000      0.000   
c16           0.000     0.000  263.480   37.640   0.000      0.000   
glycine       7.799     0.365    0.981    0.408   0.034      0.137   
glutamate  

In [6]:
# estimation des min-max AIO
"""
print("Sampling")
#sampling_V = sample(model,50, processes=4)
sampling_V = sample(model,3)
sampling_V.to_csv('allo.csv', sep=';')
end = time.time()
print("       finishing in",end - start," s")

start = time.time()
print("Min Max AIO")
Min_AIO_python, Max_AIO_python=Estimation_MinMax_AIO(model, sampling_V)
distMin=Sum_EuclidianDist_ColonneMatriceAIO(Min_AIO_python)
distMax=Sum_EuclidianDist_ColonneMatriceAIO(Max_AIO_python)


end = time.time()
print("       finishing in",end - start," s")
#print(Min_AIO_python)
Min_AIO_python.to_csv('MinAIO_FirstSample.csv', sep=';', index = False, header = False)
Max_AIO_python.to_csv('MaxAIO_FirstSample.csv', sep=';', index = False, header = False)

cpt=0
valNb=1
Tol=0.1

while (distMin >  Tol or distMax > Tol) :
    sampling_V = sample(model, valNb)
    Min_AIO_python_Pred, Max_AIO_python_Pred, Min_AIO_python_Suc, Max_AIO_python_Suc=Estimation_MinMax_AIO(model, sampling_V, True)

    
    Min_AIO_python_Suc.to_csv('MinAIO_FirstSample.csv', sep=';', index = False, header = False)
    Max_AIO_python_Suc.to_csv('MaxAIO_FirstSample.csv', sep=';', index = False, header = False)
    diffMin_SucPred=Min_AIO_python_Suc- Min_AIO_python_Pred
    diffMax_SucPred=Max_AIO_python_Suc - Max_AIO_python_Pred
    #print(diffMin_SucPrec)
    #print(Min_AIO_python_Suc)
    
    distMin=Sum_EuclidianDist_ColonneMatriceAIO(diffMin_SucPred)
    distMax=Sum_EuclidianDist_ColonneMatriceAIO(diffMax_SucPred)
    print(" dist Min    ")
    print(distMin)
    print(" dist Max    ")
    print(distMax)
    cpt+=valNb


print(Min_AIO_python_Suc)
print(Max_AIO_python_Suc)
print("cpt cpt ........")
print(cpt)

new_LOut, new_LIn, Min_AIO_python_Suc=MamaryRepartitionOrder(model, Min_AIO_python_Suc)

new_LOut, new_LIn, Max_AIO_python_Suc=MamaryRepartitionOrder(model, Max_AIO_python_Suc)


#print(Max_AIO_python)

#lecture des fichiers min-max AIO fmincon
AIO_fmincon = pd.read_csv('Max_T.csv', sep =';', header=None)
#print(AIO_fmincon)

Max_AIO_fmincon = pd.read_csv('Max_T.csv', sep =';', names =new_LIn)
Max_AIO_fmincon.index=new_LOut
#df.columns=new_LIn

print("... Max_AIO_fmincon-Max_AIO_python")
diffMax=Max_AIO_fmincon-Max_AIO_python_Suc


Min_AIO_fmincon = pd.read_csv('Min_T.csv', sep =';', names =new_LIn)
Min_AIO_fmincon.index=new_LOut
#df.columns=new_LIn

print("... Min_AIO_fmincon-Min_AIO_python")
diffMin=Min_AIO_fmincon-Min_AIO_python_Suc

#concatenation verticale
diffMaxMin = pd.concat([diffMax,diffMin], ignore_index=False)

diffMaxMin.to_csv('diffMaxMin_FminconPython.csv', index=True, header=True, sep=';')
"""


"""

# Compute one vector
#model.objective="Biomass"
model.objective = model.reactions.R19.flux_expression + model.reactions.R15.flux_expression
fba = model.optimize()
V=fba.fluxes
#FixModelFluxes(model,V)


#model.metabolites.ATP.remove_from_model()

print(fba.status)

print(fba.objective_value)

#fba.fluxes.to_json()

fba.fluxes.to_json("solution.json")

import json
#from pprint import pprint

with open("solution.json") as f:
    data = json.load(f)

for key, value in data.items():
    print(key, value)
"""




#for i in range (2,len(s.index)):
    #print(i)
    #print(s.loc[i])
    
    
#V=s.loc[0]
#AIO_matrice=PrintRepartition(model,V)






#RescaleModel(model,0.5)

'\n\n# Compute one vector\n#model.objective="Biomass"\nmodel.objective = model.reactions.R19.flux_expression + model.reactions.R15.flux_expression\nfba = model.optimize()\nV=fba.fluxes\n#FixModelFluxes(model,V)\n\n\n#model.metabolites.ATP.remove_from_model()\n\nprint(fba.status)\n\nprint(fba.objective_value)\n\n#fba.fluxes.to_json()\n\nfba.fluxes.to_json("solution.json")\n\nimport json\n#from pprint import pprint\n\nwith open("solution.json") as f:\n    data = json.load(f)\n\nfor key, value in data.items():\n    print(key, value)\n'